In [5]:
import collections
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

Using the MNIST train and Test Data

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.datasets import mnist

# Load the MNIST dataset
(x_train, y_train), (x_test,y_test) = mnist.load_data()

x_train = x_train / 255.0
y_train = y_train.astype(np.int32)
y_train = to_categorical(y_train, 10)


x_test = x_test / 255.0
y_test = y_test.astype(np.int32)
y_test = to_categorical(y_test, 10)

Here We are creating a Global Model Architecture

In [7]:
global_model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(28, 28)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

Metal device set to: Apple M1


In [8]:
global_model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

The Hyper-Parameters and Creating a IID

In [9]:
NUM_CLIENTS = 10
EPOCHS_PER_ROUND = 5
BATCH_SIZE = 20

In [10]:
x_train_clients = np.array_split(x_train,NUM_CLIENTS)
y_train_clients = np.array_split(y_train,NUM_CLIENTS)

x_test_clients = np.array_split(x_test,NUM_CLIENTS)
y_test_clients = np.array_split(y_test,NUM_CLIENTS)

In [11]:
client_models = []
for i in range(NUM_CLIENTS):
    local_model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    local_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
    local_model.fit(x_train_clients[i],y_train_clients[i],epochs=EPOCHS_PER_ROUND, batch_size=BATCH_SIZE,verbose=0)
    acc = local_model.evaluate(x_test_clients[i],y_test_clients[i])
    print(acc)
    client_models.append(local_model)

2023-08-23 22:56:20.739790: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


32/32 [==============================] - 0s 11ms/step - loss: 0.2399 - accuracy: 0.9240
[0.23991699516773224, 0.9240000247955322]
32/32 [==============================] - 0s 7ms/step - loss: 0.3516 - accuracy: 0.8990
[0.3516087830066681, 0.8989999890327454]
32/32 [==============================] - 0s 7ms/step - loss: 0.2610 - accuracy: 0.9260
[0.26102498173713684, 0.9259999990463257]
32/32 [==============================] - 0s 7ms/step - loss: 0.3211 - accuracy: 0.9060
[0.3210887610912323, 0.906000018119812]
32/32 [==============================] - 0s 7ms/step - loss: 0.2946 - accuracy: 0.9170
[0.29457181692123413, 0.9169999957084656]
32/32 [==============================] - 0s 10ms/step - loss: 0.1999 - accuracy: 0.9430
[0.19994628429412842, 0.9430000185966492]
32/32 [==============================] - 0s 8ms/step - loss: 0.2084 - accuracy: 0.9350
[0.20835934579372406, 0.9350000023841858]
32/32 [==============================] - 0s 7ms/step - loss: 0.1217 - accuracy: 0.9670
[0.12167900

In [13]:
# Performing Federated Averaging
weights = global_model.get_weights()
for i in range(len(weights)):
    for j in range(NUM_CLIENTS):
        client_weights = client_models[j].get_weights()
        weights[i] += client_weights[i]/NUM_CLIENTS

global_model.set_weights(weights)

In [14]:
acc = global_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 2s 8ms/step - loss: 1.3628 - accuracy: 0.5194
[1.3628222942352295, 0.5194000005722046]


In [15]:
# Train the Fed AVG model
fed_model = global_model
fed_model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
fed_model.fit(x_train,y_train,epochs=5,batch_size=20)

Epoch 1/5
3000/3000 [==============================] - 28s 9ms/step - loss: 0.1964 - accuracy: 0.9417
Epoch 2/5
3000/3000 [==============================] - 28s 9ms/step - loss: 0.1017 - accuracy: 0.9690
Epoch 3/5
3000/3000 [==============================] - 28s 9ms/step - loss: 0.0705 - accuracy: 0.9785
Epoch 4/5
3000/3000 [==============================] - 28s 9ms/step - loss: 0.0519 - accuracy: 0.9833
Epoch 5/5
3000/3000 [==============================] - 28s 9ms/step - loss: 0.0414 - accuracy: 0.9870


In [16]:
acc = fed_model.evaluate(x_test,y_test)
print(acc)

313/313 [==============================] - 3s 8ms/step - loss: 0.0826 - accuracy: 0.9772
[0.08262024819850922, 0.9771999716758728]
